In [1]:
pip install tensorflow keras opencv-python transformers pandas numpy nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00


In [6]:
!pip install openpyxl xlrd

In [9]:
# Import necessary libraries
import pandas as pd
import os
import nltk
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
import os

excel_folder = '../datasets/mimic-iii-clinical-database-demo-1.4/'
files = os.listdir(excel_folder)
print("Files in the directory:")
print(files)

Files in the directory:
['ADMISSIONS.csv', 'CALLOUT.csv', 'CAREGIVERS.csv', 'CHARTEVENTS.csv', 'CPTEVENTS.csv', 'DATETIMEEVENTS.csv', 'DIAGNOSES_ICD.csv', 'DRGCODES.csv', 'D_CPT.csv', 'D_ICD_DIAGNOSES.csv', 'D_ICD_PROCEDURES.csv', 'D_ITEMS.csv', 'D_LABITEMS.csv', 'ICUSTAYS.csv', 'INPUTEVENTS_CV.csv', 'INPUTEVENTS_MV.csv', 'LABEVENTS.csv', 'LICENSE.txt', 'MICROBIOLOGYEVENTS.csv', 'NOTEEVENTS.csv', 'OUTPUTEVENTS.csv', 'PATIENTS.csv', 'PRESCRIPTIONS.csv', 'PROCEDUREEVENTS_MV.csv', 'PROCEDURES_ICD.csv', 'SERVICES.csv', 'SHA256SUMS.txt', 'TRANSFERS.csv']


In [28]:
import pandas as pd
import os

# Path to your MIMIC-III dataset files
excel_folder = '../datasets/mimic-iii-clinical-database-demo-1.4/'

# Function to load and analyze files
def load_and_analyze_files(folder_path):
    dataframes = {}
    
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith('.csv') or file.endswith('.xlsx'):
            print(f"Attempting to read {file_path}...")
            try:
                if file.endswith('.csv'):
                    df = pd.read_csv(file_path)  # Read CSV files
                elif file.endswith('.xlsx'):
                    df = pd.read_excel(file_path, engine='openpyxl')  # Read Excel files
                
                # Store the DataFrame in a dictionary
                dataframes[file] = df
                
                # Print the shape and first few rows
                print(f"Successfully read {file_path}, shape: {df.shape}")
                print(f"First few rows of {file}:\n{df.head()}\n")
                
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    
    return dataframes

# Load and analyze the files
dataframes = load_and_analyze_files(excel_folder)

# Determine which files are useful based on the output
print("\nFiles that may be useful for analysis:")
for filename, df in dataframes.items():
    # Basic conditions to determine usefulness (you can customize this)
    if not df.empty:  # Only consider non-empty DataFrames
        if 'diagnosis' in df.columns or 'icd9_code' in df.columns:  # Check for diagnosis-related columns
            print(f"- {filename}: Contains diagnosis data.")
        if 'text' in df.columns or 'description' in df.columns:  # Check for notes or descriptions
            print(f"- {filename}: Contains clinical notes or descriptions.")
        if 'hospital_expire_flag' in df.columns:  # Check for hospital expiration flag
            print(f"- {filename}: Contains hospital expiration flag.")

# Summary of useful files
print("\nBased on the analysis, the following files are recommended for further analysis:")

Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv...
Successfully read ../datasets/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv, shape: (129, 19)
First few rows of ADMISSIONS.csv:
   row_id  subject_id  hadm_id            admittime            dischtime  \
0   12258       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00   
1   12263       10011   105331  2126-08-14 22:32:00  2126-08-28 18:59:00   
2   12265       10013   165520  2125-10-04 23:36:00  2125-10-07 15:13:00   
3   12269       10017   199207  2149-05-26 17:19:00  2149-06-03 18:42:00   
4   12270       10019   177759  2163-05-14 20:43:00  2163-05-15 12:00:00   

             deathtime admission_type         admission_location  \
0                  NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1  2126-08-28 18:59:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
2  2125-10-07 15:13:00      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3                  NaN      EMERGENCY       EMERGEN

C:\Users\ADITYA SINGH\AppData\Local\Temp\ipykernel_176488\3123094874.py:17: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Read CSV files


Successfully read ../datasets/mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv, shape: (758355, 15)
First few rows of CHARTEVENTS.csv:
    row_id  subject_id  hadm_id  icustay_id  itemid            charttime  \
0  5279021       40124   126179    279554.0  223761  2130-02-04 04:00:00   
1  5279022       40124   126179    279554.0  224695  2130-02-04 04:25:00   
2  5279023       40124   126179    279554.0  220210  2130-02-04 04:30:00   
3  5279024       40124   126179    279554.0  220045  2130-02-04 04:32:00   
4  5279025       40124   126179    279554.0  220179  2130-02-04 04:32:00   

             storetime   cgid      value   valuenum  valueuom  warning  error  \
0  2130-02-04 04:35:00  19085       95.9       95.9        ?F      0.0    0.0   
1  2130-02-04 05:55:00  18999  2222221.7  2222221.7     cmH2O      0.0    0.0   
2  2130-02-04 04:43:00  21452       15.0       15.0  insp/min      0.0    0.0   
3  2130-02-04 04:43:00  21452       94.0       94.0       bpm      0.0    0.0   

C:\Users\ADITYA SINGH\AppData\Local\Temp\ipykernel_176488\3123094874.py:17: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)  # Read CSV files


Successfully read ../datasets/mimic-iii-clinical-database-demo-1.4/INPUTEVENTS_MV.csv, shape: (13224, 31)
First few rows of INPUTEVENTS_MV.csv:
   row_id  subject_id  hadm_id  icustay_id            starttime  \
0  118897       42367   139932      250305  2147-10-29 16:45:00   
1  118898       42367   139932      250305  2147-10-20 13:17:00   
2  118899       42367   139932      250305  2147-10-29 03:23:00   
3  118900       42367   139932      250305  2147-10-22 22:00:00   
4  118901       42367   139932      250305  2147-10-16 06:21:00   

               endtime  itemid       amount amountuom        rate  ...  \
0  2147-10-29 16:46:00  225799    60.000000        ml         NaN  ...   
1  2147-10-20 13:18:00  223258    10.000000     units         NaN  ...   
2  2147-10-29 03:53:00  226089    99.999999        ml  199.999998  ...   
3  2147-10-22 22:01:00  225799    40.000000        ml         NaN  ...   
4  2147-10-17 06:10:00  225936  1309.899995        ml   54.999300  ...   

  totala

In [29]:
import pandas as pd
import os

# Path to your MIMIC-III dataset files
excel_folder = '../datasets/mimic-iii-clinical-database-demo-1.4/'

# Define the required columns for each file
required_columns = {
    'ADMISSIONS.csv': ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'diagnosis', 'hospital_expire_flag'],
    'DIAGNOSES_ICD.csv': ['subject_id', 'hadm_id', 'icd9_code'],
    'CHARTEVENTS.csv': ['subject_id', 'hadm_id', 'itemid', 'value', 'charttime'],
    'CPTEVENTS.csv': ['subject_id', 'hadm_id', 'cpt_cd', 'description'],
    'D_ICD_DIAGNOSES.csv': ['icd9_code', 'short_title', 'long_title'],
    'D_ICD_PROCEDURES.csv': ['icd9_code', 'short_title', 'long_title'],
    'PROCEDURES_ICD.csv': ['subject_id', 'hadm_id', 'icd9_code'],
    'LABEVENTS.csv': ['subject_id', 'hadm_id', 'itemid', 'value', 'charttime']  # Optional
}

# Create a folder for final CSV files if it doesn't exist
final_folder = '../datasets/mimic-iii-clinical-database-demo-1.4/final_files/'
os.makedirs(final_folder, exist_ok=True)

# Function to load and extract relevant columns
def load_and_save_files(folder_path, required_columns):
    for file, columns in required_columns.items():
        file_path = os.path.join(folder_path, file)
        print(f"Attempting to read {file_path}...")
        try:
            if file.endswith('.csv'):
                df = pd.read_csv(file_path)
            elif file.endswith('.xlsx'):
                df = pd.read_excel(file_path, engine='openpyxl')
                
            # Extract the required columns
            df_required = df[columns].copy()  # Use .copy() to avoid SettingWithCopyWarning
            output_file_path = os.path.join(final_folder, file)  # Save in final folder
            df_required.to_csv(output_file_path, index=False)
            print(f"Saved {file} to {output_file_path}, shape: {df_required.shape}")
        
        except Exception as e:
            print(f"Error reading or saving {file_path}: {e}")

# Load and save the required columns to final CSV files
load_and_save_files(excel_folder, required_columns)

Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv...
Saved ADMISSIONS.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/ADMISSIONS.csv, shape: (129, 6)
Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/DIAGNOSES_ICD.csv...
Saved DIAGNOSES_ICD.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/DIAGNOSES_ICD.csv, shape: (1761, 3)
Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/CHARTEVENTS.csv...


C:\Users\ADITYA SINGH\AppData\Local\Temp\ipykernel_176488\3602635969.py:30: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Saved CHARTEVENTS.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/CHARTEVENTS.csv, shape: (758355, 5)
Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/CPTEVENTS.csv...
Saved CPTEVENTS.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/CPTEVENTS.csv, shape: (1579, 4)
Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/D_ICD_DIAGNOSES.csv...
Saved D_ICD_DIAGNOSES.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/D_ICD_DIAGNOSES.csv, shape: (14567, 3)
Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/D_ICD_PROCEDURES.csv...
Saved D_ICD_PROCEDURES.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/D_ICD_PROCEDURES.csv, shape: (3882, 3)
Attempting to read ../datasets/mimic-iii-clinical-database-demo-1.4/PROCEDURES_ICD.csv...
Saved PROCEDURES_ICD.csv to ../datasets/mimic-iii-clinical-database-demo-1.4/final_files/PROCEDURES_ICD.csv, shape: (506, 3)
Attempting to read .

In [34]:
import pandas as pd
import numpy as np

# Load the final CSV files
admissions_df = pd.read_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/ADMISSIONS.csv')
diagnoses_df = pd.read_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/DIAGNOSES_ICD.csv')
chartevents_df = pd.read_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/CHARTEVENTS.csv')
cptevents_df = pd.read_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/CPTEVENTS.csv')
patients_df = pd.read_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/PATIENTS.csv')
prescriptions_df = pd.read_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/PRESCRIPTIONS.csv')

# Display the shapes of the dataframes to understand the data size
print(f'Admittances: {admissions_df.shape}')
print(f'Diagnoses: {diagnoses_df.shape}')
print(f'Chartevents: {chartevents_df.shape}')
print(f'Cptevents: {cptevents_df.shape}')
print(f'Patients: {patients_df.shape}')
print(f'Prescriptions: {prescriptions_df.shape}')

# Convert dates to datetime format where applicable
admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'], errors='coerce')
admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'], errors='coerce')
patients_df['dob'] = pd.to_datetime(patients_df['dob'], errors='coerce')
patients_df['dod'] = pd.to_datetime(patients_df['dod'], errors='coerce')

# Check for missing values and handle them
print("\nMissing values in admissions_df:")
print(admissions_df.isnull().sum())

print("\nMissing values in patients_df:")
print(patients_df.isnull().sum())

# Example: Fill missing values or drop rows/columns based on the requirements
# Here we can fill missing values with a placeholder (or you can choose to drop them)
admissions_df.fillna({'diagnosis': 'Unknown'}, inplace=True)
patients_df.fillna({'gender': 'Unknown'}, inplace=True)

# Ensure that the relevant columns are selected for the recommendation system
required_columns_admissions = ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'diagnosis', 'hospital_expire_flag']
required_columns_patients = ['subject_id', 'gender', 'dob', 'expire_flag']

# Create final DataFrames with required columns
final_admissions_df = admissions_df[required_columns_admissions]
final_patients_df = patients_df[required_columns_patients]

# Display the head of the final DataFrames
print("\nFinal Admissions DataFrame:")
print(final_admissions_df.head())

print("\nFinal Patients DataFrame:")
print(final_patients_df.head())

# Optionally save the final DataFrames to CSV for future use
final_admissions_df.to_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/final_admissions.csv', index=False)
final_patients_df.to_csv('../datasets/mimic-iii-clinical-database-demo-1.4/final_files/final_patients.csv', index=False)

print("\nFinal CSV files have been saved.")

Admittances: (129, 6)
Diagnoses: (1761, 3)
Chartevents: (758355, 5)
Cptevents: (1579, 4)
Patients: (100, 8)
Prescriptions: (10398, 19)

Missing values in admissions_df:
subject_id              0
hadm_id                 0
admittime               0
dischtime               0
diagnosis               0
hospital_expire_flag    0
dtype: int64

Missing values in patients_df:
row_id          0
subject_id      0
gender          0
dob             0
dod             0
dod_hosp       30
dod_ssn        23
expire_flag     0
dtype: int64

Final Admissions DataFrame:
   subject_id  hadm_id           admittime           dischtime  \
0       10006   142345 2164-10-23 21:09:00 2164-11-01 17:15:00   
1       10011   105331 2126-08-14 22:32:00 2126-08-28 18:59:00   
2       10013   165520 2125-10-04 23:36:00 2125-10-07 15:13:00   
3       10017   199207 2149-05-26 17:19:00 2149-06-03 18:42:00   
4       10019   177759 2163-05-14 20:43:00 2163-05-15 12:00:00   

             diagnosis  hospital_expire_flag  
